In [1]:
import os
filename = 'Abyssinian_1.jpg'
extension = os.path.splitext(filename)[1]
label = filename.replace(extension, '')
print(label)

Abyssinian_1


In [2]:
import os
path_input = os.path.join('data/pets/images')
list_filename = os.listdir(path_input)

list_filenames = os.listdir(path_input)
list_file = []
for filename in list_filenames:
    if ('newfoundland' in filename) or ('pomeranian' in filename):
        label = 0 # dog
    elif ('Abyssinian' in filename) or ('Bombay' in filename):
        label = 1 # cat
    else:
        continue
    list_file.append([filename, label, filename.split('_')[0]])
print(list_file[0])
# ['Abyssinian_1.jpg', 1, 'Abyssinian']

['pomeranian_39.jpg', 0, 'pomeranian']


In [3]:
from sklearn.model_selection import train_test_split

random_seed = 0

list_train, list_val = train_test_split(list_file, shuffle=True, random_state=random_seed, test_size=0.2)
list_val, list_test = train_test_split(list_val, shuffle=True, random_state=random_seed, test_size=0.5)

In [4]:
import os
from PIL import Image
import torch.utils.data as data

class MyDataset(data.Dataset):
    def __init__(self, list_file, transform=None, phase='train'):
        self.list_file = list_file
        self.transform = transform
        self.phase = phase

    def __len__(self):
        # ファイル数を返す
        return len(self.list_file)

    # def __getitem__(self, index):
    #     # 画像をPillowsで開く
    #     path_input = './data/pets/images/'
    #     path_image = os.path.join(path_input, self.list_file[index][0])
    #     pil_image = Image.open(path_image)

    #     # 画像の前処理
    #     image_transformed = self.transform(pil_image).convert('RGB')

    #     # ラベルを取得
    #     label_class = self.list_file[index][1]
    #     label_type = self.list_file[index][2]
    #     return image_transformed, label_class



    def __getitem__(self, index):
        path_input = './data/pets/images/'
        path_image = os.path.join(path_input, self.list_file[index][0])
        pil_image = Image.open(path_image).convert('RGB') # <-- ここでPILのままRGBに変換
        
        # 画像の前処理
        image_transformed = self.transform(pil_image) # <-- transformがTensorに変換する
        
        # ラベルを取得
        label_class = self.list_file[index][1]
        label_type = self.list_file[index][2]
        
        return image_transformed, label_class

In [5]:
from torchvision import transforms

class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = transforms.Compose([
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

    def __call__(self, image):
        return self.data_transform(image)

In [6]:
resize = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
transform = ImageTransform(resize, mean, std)

In [7]:
# dataset
train_dataset = MyDataset(list_train, transform=transform, phase='train')
val_dataset = MyDataset(list_val, transform=transform, phase='val')

In [8]:
# dataloader
batch_size = 8

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [9]:
# デバイスを選択
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [10]:
from torchvision import models
net = models.vgg16(weights='VGG16_Weights.IMAGENET1K_V1')
# 出力層を2つに付け替える

import torch.nn as nn
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)
net.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
import torch.nn as nn
# 損失関数の定義
criterion = nn.CrossEntropyLoss()

In [12]:
import torch.optim as optim
lr = 0.001
USE_FINE_TUNING = True
# 最適化手法の選択
if USE_FINE_TUNING:
    # 最適化手法を設定
    optimizer = optim.Adam(net.parameters(), lr=lr)
else:
    # 転移学習
    params_to_update = []
    update_param_names = ['classifier.6.weight', 'classifier.6.bias']
    for name, param in net.named_parameters():
        if name in update_param_names:
            param.requires_grad = True
            params_to_update.append(param)
        else:
            param.requires_grad = False
        optimizer = optim.Adam(params=params_to_update, lr=lr)

In [13]:
from sklearn.metrics import accuracy_score

def validation(net, device, criterion, val_dataloader):
    net.eval()
    total_loss = 0
    Y = []
    preds = []
    with tqdm(total=len(val_dataloader)) as pbar:
        pbar.set_description('validation')
        for inputs, labels in val_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            with torch.no_grad():
                outputs = net(inputs)
            loss = criterion(outputs, labels)
            _, pred = torch.max(outputs, 1) # ラベルの予想
            total_loss += loss.item() * inputs.size(0)
            # Y.extend(labels)
            # preds.extend(pred)

            Y.extend(labels.cpu().tolist())
            preds.extend(pred.cpu().tolist())

            pbar.update(1)
    return accuracy_score(y_true=Y, y_pred=preds), total_loss, Y, preds

In [ ]:
from tqdm import tqdm
import time

count = 0
iteration = 0
total_loss = 0
max_itr = 1000
val_interval = 100
path_save_logfile_train = "/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Cygnus-X_cloud/Binary_classification/path_save_logfile_train.txt"
path_save_logfile_val = "/home/cygnus/fujimoto/Cygnus-X_Molecular_Cloud_Analysis/Cygnus-X_cloud/Binary_classification/path_save_logfile_val.txt"

Y_train = []
pred_train = []
time_trainval_total_start = time.perf_counter()
with tqdm(total=max_itr) as pbar:
    pbar.set_description('training')
    while iteration < max_itr:
        for inputs, labels in train_dataloader:
            if iteration >= max_itr:
                break
            inputs = inputs.to(device)
            labels = labels.to(device)
            if count == 0:
                net.train()
                time_trainval_interval_start = time.perf_counter()
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                _, pred = torch.max(outputs, 1) # ラベルの予測

                # バックプロパゲーション
                loss.backward()
                optimizer.step()

            # カウント
            count += 1
            iteration += 1
            # 損失計算
            total_loss += loss.item() * inputs.size(0)
            # スコア計算用
            # Y_train.extend(labels)
            # pred_train.extend(pred)
            Y_train.extend(labels.cpu().tolist())
            pred_train.extend(pred.cpu().tolist())

            if count == val_interval:
                time_trainval_interval_end = time.perf_counter()
                time_trainval_interval = time_trainval_interval_end - time_trainval_interval_start
                total_loss = total_loss / val_interval
                # validation
                acc_score, loss_val, Y, preds = validation(net, device=device, criterion=criterion, val_dataloader=val_dataloader)
                # save log
                ## training
                with open(path_save_logfile_train, 'a') as logfile:
                    logfile.write('{},{},{},{}\n'.format(iteration, time_trainval_interval, total_loss, accuracy_score(y_true=Y_train, y_pred=pred_train)))
                ## validation
                with open(path_save_logfile_val, 'a') as logfile:
                    logfile.write('{},{},{},{}\n'.format(iteration, time_trainval_interval, loss_val, acc_score))

                # 結果の描画

                # reset
                count = 0
                Y_train = []
                pred_train = []

            pbar.update(1)


training:  96%|████████████████████████████████████████████████████████████████████████   | 961/1000 [00:52<00:02, 19.17it/s]